In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
from time import sleep # this is to create a time delay

# 1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [ ]:
URL = 'https://www.billboard.com/charts/hot-100/'

response = requests.get(URL)
response

In [ ]:
soup = BS(response.text)
#soup

In [ ]:
soup.find('h3').text

In [ ]:
songs = soup.findAll('h3')
titles = []
for song in songs:
    meta_data = str(song)
    #print(meta_data)
    
    pattern = re.compile('a-no-trucate')
    
    if pattern.search(meta_data):
        song_cleaned = re.subn('[\n\t]', '', song.text)[0]
        #print(f'Result: "{song_cleaned}" is a song!\n')
        titles.append(re.subn('[\n\t]', '', song.text)[0])
    #else:
    #    print('Result: It is not a song\n')

In [ ]:
titles

In [ ]:
len(titles)

### Now to collect the other categories

In [ ]:
spans = soup.findAll('span')

In [ ]:
print(len(spans))
spans

In [ ]:
pattern = re.compile('lrv-u-padding-tb-050@mobile-max')
span_filter = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        span_filter.append(span)

In [ ]:
span_filter

In [ ]:
len(span_filter)

There are 3 entries of data for each song that we are not using. As such, I'll use a loop to filter which data to use and where we'll put it.

In [ ]:
i = 1
last_week = []
peak_pos = []
weeks_on_chart = []


for span in span_filter:
    if i%6 == 1:
        last_week.append(span.text.strip('\n\t'))
    elif i%6 == 2:
        peak_pos.append(span.text.strip('\n\t'))
        print()
    elif i%6 == 3:
        weeks_on_chart.append(span.text.strip('\n\t'))

    i += 1
#end

### Let me explain this loop

When I was cycling through the instances of the elements found in span_filter, I noticed that every 4th, 5th, and 6th entry was not visible on the website, meaning I had to disregard it. What this function is doing is just that: disregarding every 4th, 5th, and 6th element.

This is done using %. This is commonly referred to as "modulus" to math people like myself, but don't worry if that term scares you. The concept is very easy: it gives you remainders when you divide.

For instance, 5/2 = 2.5, but 5%2 = 1. This is because 2 goes into 5 a total of 2 times, but has a remainder of 1. Since the remainder is 1, that is what % returns. Weird right? Who would've thought that dividing with remainders would be used anywhere outside of elementary school?

Anyway, since there are 3 data points I need and 3 that I don't, I created an iterable variable (which is 'i') to keep track of which data point I was on. The reason I use i%6 is beacuse each song has 6 data points (again, with 3 that I need and 3 that I need to disregard). As such, I was able to succesfully filter out the ones I didn't need and assign the ones I did need.

In [ ]:
print(len(last_week))
last_week

In [ ]:
print(len(peak_pos))
peak_pos

In [ ]:
print(len(weeks_on_chart))
weeks_on_chart

### Creating the Data Frame to add to later

In [ ]:
songs_df = pd.DataFrame(titles)
songs_df.columns = ['Title']
songs_df

### Finding the artists for each song

In [ ]:
pattern = re.compile('a-no-trucate')
artists = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        artists.append(span.text.strip('\n\t'))

In [ ]:
artists

### Finding rankings of songs

In [ ]:
pattern = re.compile('u-letter-spacing-0080@tablet')
ranks = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        ranks.append(span.text.strip('\n\t'))

In [ ]:
ranks

In [ ]:
songs_df['Rank'] = ranks
songs_df = songs_df.set_index('Rank')
songs_df

In [ ]:
songs_df['Artist(s)'] = artists
songs_df['Last Week'] = last_week
songs_df['Peak Position'] = peak_pos
songs_df['Weeks on Chart'] = weeks_on_chart
songs_df